In [1]:
import pdfplumber
import pandas as pd

In [2]:
PDF_PATH = "DP.pdf"

In [5]:
tables = []
with pdfplumber.open(PDF_PATH) as pdf:
        for page in pdf.pages:
            t = page.extract_tables()
            if t:
                tables.extend(t)    
len(tables)                

172

In [10]:
final_table = []
for table in tables:
    for row in table:
        if row[1] == "bakalavriat" or row[1] == "magistratura":
            final_table.append(row)

df = pd.DataFrame(final_table, columns=['Nömrə', 'Təhsil səviyyəsi', 'Ölkə', 'Universitet', 'Təhsil proqramı'])
df.head()

,Nömrə,Təhsil səviyyəsi,Ölkə,Universitet,Təhsil proqramı
0,1,bakalavriat,Almaniya Federativ Respublikası,"""Technical University of Munich""","""Architecture"""
1,2,bakalavriat,Almaniya Federativ Respublikası,"""Technical University of Munich""","""Landscape Architecture and Landscape Planning"""
2,3,bakalavriat,Birləşmiş Ərəb Əmirlikləri,"""Abu Dhabi University""","""Architecture"""
3,4,bakalavriat,Birləşmiş Ərəb Əmirlikləri,"""Abu Dhabi University""","""Interior Design"""
4,5,bakalavriat,Birləşmiş Ərəb Əmirlikləri,"""United Arab Emirates University""","""Architectural Engineering"""


In [11]:
df.shape

(4405, 5)

In [14]:
# let's rename the columns to English and add ID column instead of the first column
df.columns = ['id', 'education_level', 'country', 'university', 'program']
df['id'] = range(1, len(df) + 1)
df.head()

,id,education_level,country,university,program
0,1,bakalavriat,Almaniya Federativ Respublikası,"""Technical University of Munich""","""Architecture"""
1,2,bakalavriat,Almaniya Federativ Respublikası,"""Technical University of Munich""","""Landscape Architecture and Landscape Planning"""
2,3,bakalavriat,Birləşmiş Ərəb Əmirlikləri,"""Abu Dhabi University""","""Architecture"""
3,4,bakalavriat,Birləşmiş Ərəb Əmirlikləri,"""Abu Dhabi University""","""Interior Design"""
4,5,bakalavriat,Birləşmiş Ərəb Əmirlikləri,"""United Arab Emirates University""","""Architectural Engineering"""


In [15]:
df.to_csv("DP.csv", index=False)